In [1]:
from gurobipy import Model, quicksum, GRB
import pandas as pd

In [2]:
Cust = set(range(1,12))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2,3,4,5}

In [3]:
T = 8
Period = list(range(1, T+1))

df = pd.read_csv('demand.csv', header=None)

demand = {(j,t):df[t-1][j-1] for j in Cust for t in Period}

transport = {1:15.2, 2:14.8, 3:15.8, 4: 12.6, 5:13.4, 6:15.8, 7:16.2, 8:16.4, 9:16.4, 10:14.6, 11:15.4}
delivery = {(6,4):30, (3,6):28, (7,11):42, (1,7):22, (11,1):18, (2,4):18, (11,10):26, (8,5):34, (9,1):24, (9,3):12, (1,6):18, (9,5):34, (2,5):22, (4,11):16, (3,1):16, (2,1):12, (9,2):14, (5,1):8, (3,9):12, (2,3):8, (9,4):24, (5,3):22, (2,2):0, (10,6):32, (5,7):30, (11,7):42, (11,3):28, (7,8):12, (7,3):22, (8,3):16, (8,2):12, (7,1):22, (3,4):12, (10,7):28, (5,4):16, (11,6):34, (1,8):24, (4,9):24, (5,8):34, (1,9):24, (11,2):30, (1,1):0, (7,2):14, (7,9):20, (4,6):30, (2,11):30, (2,10):14, (4,3):12, (4,8):28, (9,11):40, (9,10):16, (4,1):12, (11,5):12, (8,1):24, (10,11):26, (10,4):10, (3,7):22, (7,10):28, (4,5):16, (6,10):32, (8,8):0, (6,7):16, (6,1):18, (11,8):42, (6,9):32, (3,11):28, (6,3):28, (1,2):12, (5,11):12, (11,4):16, (6,5):22, (1,3):16, (10,5):24, (2,9):14, (9,7):20, (2,8):12, (3,2):8, (9,9):0, (10,10):0, (11,9):40, (2,6):20, (11,11):0, (9,6):32, (9,8):8, (5,9):34, (7,5):30, (8,6):26, (2,7):14, (7,7):0, (3,5):22, (5,6):22, (1,5):8, (5,5):0, (10,3):6, (7,4):30, (10,2):14, (8,7):12, (4,2):18, (3,8):16, (1,10):18, (3,10):6, (7,6):16, (5,2):22, (8,11):42, (1,4):12, (6,11):34, (5,10):24, (8,9):8, (4,7):30, (4,10):10, (4,4):0, (3,3):0, (6,6):0, (10,1):18, (8,10):22, (8,4):28, (1,11):18, (10,9):16, (6,8):26, (6,2):20, (10,8):22}

dc_run = {(6,1):3500, (2,5):2000, (7,3):2100, (8,3):1920, (6,3):2100, (8,2):2560, (7,1):3500, (1,2):4800, (11,4):1350, (11,1):3000, (5,4):1935, (10,1):3000, (4,2):3600, (10,3):1800, (8,5):1280, (10,5):1200, (5,2):3440, (9,1):3000, (11,2):2400, (8,4):1440, (3,2):3840, (6,5):1400, (1,1):6000, (7,2):2800, (9,3):1800, (9,5):1200, (7,4):1575, (6,4):1575, (10,2):2400, (7,5):1400, (2,4):2250, (3,1):4800, (2,1):5000, (3,4):2160, (9,2):2400, (5,1):4300, (4,3):2700, (2,3):3000, (4,4):2025, (9,4):1350, (5,3):2580, (4,1):4500, (2,2):4000, (3,3):2880, (11,5):1200, (5,5):1720, (8,1):3200, (3,5):1920, (1,4):2700, (4,5):1800, (6,2):2800, (10,4):1350, (1,5):2400, (11,3):1800, (1,3):3600}
dc_stock = 5
dc_ub = {1:120, 2:100, 3:80, 4:40, 5:30}
dc_new = 1500
dc_cancel = 1000

cust_stock = 30
cust_out = 200

dc_disposal = 20

In [4]:
model = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [5]:
X, x, y, sD, sC, z, w, o, d = {}, {}, {}, {}, {}, {}, {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="I", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="I", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        y[i,ct,0] = model.addVar(vtype="B", ub=0, name=f'y[{i},{ct},{0}]')
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

for i in DC:
    sD[i,0] = model.addVar(vtype="I", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="I", name=f'sD[{i},{t}]')
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

for j in Cust:
    sC[j,0] = model.addVar(vtype="I", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="I", name=f'sC[{j},{t}]')
        o[j,t] = model.addVar(vtype="I", name=f'o[{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            z[i,ct,t] = model.addVar(vtype="B", name=f'z[{i},{ct},{t}]')
            w[i,ct,t] = model.addVar(vtype="B", name=f'w[{i},{ct},{t}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons, DC_UB_Cons, DC_Connect_Cons = {}, {}, {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t] - o[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t] + d[i,t]
        )

for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )

for i in DC:
    for ct in CT:
        for t in Period:
            DC_Connect_Cons[i,ct,t] = model.addConstr(
                y[i,ct,t] - y[i,ct,t-1]
                ==
                z[i,ct,t] - w[i,ct,t]
            )
    
model.update()

In [7]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock * sD[i,t] for i in DC for t in Period) +
    quicksum(cust_stock * sC[j,t] for j in Cust for t in Period) +
    quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(cust_out * o[j,t] for j in Cust for t in Period) +
    quicksum(dc_disposal * d[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

Optimize a model with 1760 rows, 2805 columns and 11176 nonzeros
Variable types: 0 continuous, 2805 integer (1375 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 354200.00000
Presolve removed 132 rows and 220 columns
Presolve time: 0.02s
Presolved: 1628 rows, 2585 columns, 10483 nonzeros
Variable types: 0 continuous, 2585 integer (1210 binary)

Root relaxation: objective 8.113753e+04, 1265 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 81137.5261    0  145 354200.000 81137.5261  77.1%     -    0s
H    0     0                    288297.20000 81137.5261  71.9%     -    0s
H    0     0                    216580.80000 81137.5261  62.5%     -    0s
H    0     0                    177031.80000 8113

In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[7,1] 80.0
X[7,2] 80.0
X[7,3] 80.0
X[7,4] 72.0
X[7,5] 78.0
X[7,6] 71.0
X[7,7] 80.0
X[7,8] 80.0
X[9,5] 59.0
X[9,6] 80.0
X[9,7] 80.0
X[9,8] 80.0
X[10,1] 80.0
X[10,2] 80.0
X[10,3] 80.0
X[10,4] 80.0
X[10,5] 80.0
X[10,6] 80.0
X[10,7] 80.0
X[10,8] 80.0
X[11,6] 57.0
X[11,7] 68.0
X[11,8] 80.0
x[11,11,6] 32.0
x[11,11,7] 40.0
x[11,11,8] 41.0
x[11,5,6] 25.0
x[11,5,7] 14.0
x[11,5,8] 19.0
x[7,6,1] 20.0
x[7,6,2] 16.0
x[7,6,3] 17.0
x[7,6,4] 10.0
x[7,6,5] 22.0
x[7,6,6] 33.0
x[7,6,7] 36.0
x[7,6,8] 30.0
x[9,9,5] 30.0
x[9,9,6] 47.0
x[9,9,7] 39.0
x[9,9,8] 34.0
x[7,8,1] 16.0
x[7,8,2] 18.0
x[7,8,3] 25.0
x[7,8,4] 10.0
x[7,8,6] 2.0
x[10,3,1] 12.0
x[10,3,2] 16.0
x[10,3,3] 6.0
x[10,3,4] 11.0
x[10,3,5] 6.0
x[10,3,6] 12.0
x[10,3,7] 25.0
x[10,3,8] 16.0
x[7,2,1] 12.0
x[7,2,2] 9.0
x[7,2,3] 6.0
x[7,2,4] 14.0
x[7,2,5] 15.0
x[10,9,1] 8.0
x[10,9,2] 16.0
x[10,9,3] 14.0
x[11,4,8] 9.0
x[10,4,1] 8.0
x[10,4,2] 8.0
x[10,4,3] 8.0
x[10,4,4] 9.0
x[10,4,5] 8.0
x[10,4,6] 14.0
x[10,4,7] 7.0
x[10,4,8] 3.0
x[7,1,1] 5.0
x[10,10,1] 26

In [12]:
model.ObjVal

93384.8